In [1]:
from src.test_trader import Trader
import pandas as pd


In [2]:
df = pd.read_csv('src/input1016.csv')
df2 = pd.read_csv('res/origin1016.csv')

In [3]:
test_bars = df[-180:]
test_xaus = df2[-180:]

In [4]:
xaus = test_xaus['xau']

In [5]:
dates = test_xaus['date']

In [6]:
me = Trader()

In [7]:
for i in range(10381,10380+len(xaus)):
    price = xaus[i]
    time = dates[i]
    pre_price = xaus[i-1]
    if price>pre_price:
        if price<pre_price*0.98: # 如果涨幅达2%，平空仓，不开空仓，如果涨了但是不足2%，开多仓，不用平空仓
            print(str(time)+' price went up too high, stop shorting!')
            if me.put_position > 0:
                me.buy_close(me.put_position,price,time)
                print("I'am running away, bye!")
            else:
                print('I have no short position, take it easy.')
        else:
            me.buy_open(5, price, time)
    else:
        if price>pre_price*1.02:# 如果跌幅达2%，平多仓，不开空仓，如果跌了但是不足2%，开空仓，不用平多仓
            print(str(time)+' price went up too low, stop longing!')
            if me.call_position>0:
                me.sell_close(me.call_position,price,time)
                print("I'am running away, bye!")
            else:
                print("I have no long position, take it easy")
        else:
            me.sell_open(5, price, time)
    me.settle(price, time)


In [8]:
me.records.to_csv('records_reverse_2.csv', index=False)
me2 = Trader()

In [9]:
for i in range(10381,10380+len(xaus)):
    price = xaus[i]
    time = dates[i]
    pre_price = xaus[i-1]
    if price>pre_price:
        if price>pre_price*1.02: # 如果涨幅达2%，平空仓，不开空仓，如果涨了但是不足2%，开多仓，不用平空仓
            print(str(time)+' price went up too high, stop shorting!')
            if me2.put_position > 0:
                me2.buy_close(me2.put_position,price,time)
                print("I'am running away, bye!")
            else:
                print('I have no short position, take it easy.')
        else:
            me2.buy_open(5, price, time)
    else:
        if price<pre_price*0.98:# 如果跌幅达2%，平多仓，不开空仓，如果跌了但是不足2%，开空仓，不用平多仓
            print(str(time)+' price went up too low, stop longing!')
            if me2.call_position>0:
                me2.sell_close(me2.call_position,price,time)
                print("I'am running away, bye!")
            else:
                print("I have no long position, take it easy")
        else:
            me2.sell_open(5, price, time)
    me2.settle(price, time)



In [10]:
me2.records

In [1]:
me2.records.to_csv('records_normal_2.csv', index=False)

# 从这开始真正的实验

In [14]:
import keras as k
from src.test_trader import Trader
import pandas as pd
import numpy as np

df = pd.read_csv('res/input1117.csv')
df['date'] = pd.to_datetime(df.date).apply(lambda x:x.timestamp()*1e3)
def ModelPredict(input):
    model = k.models.load_model('4NN.model')# 加载保存下来的模型
    return model.predict(input)# 直接用加载出来的模型，对输入 input 进行预测

In [15]:
test_input = df.iloc[10000:, 2:-1].copy()

In [16]:
test_output = df.iloc[10000:, 0:2].copy()
test_output
xaus = list(test_output['xau'])
dates = list(test_output['date'])
xaus[1]

In [17]:
model = k.models.load_model('4NN.model')

In [18]:
model.predict(test_input.iloc[1:2,:])[0][0]

In [29]:
from highcharts import Highchart
from highcharts import Highstock
The_Graph = Highstock(width=1000, height=600)
The_Graph.add_data_set([(dates[x],xaus[x]) for x in range(0,len(xaus))], 'line', name='XAU', yAxis=0, color='black')
The_Graph.set_dict_options({
    'title' : {
        'text' : 'Strategy 1'
    },    
    'xAxis': {
        'type': 'datetime',
        'labels': {
            'step': 1, 
            'formatter': '''function () {
                return Highcharts.dateFormat('%Y-%m-%d', this.value);
            }'''
        }},

    'yAxis': [{ 
        'height': '100%',
        'lineWidth': 2,
    }],
    'legend': {
        'enabled': True
    },
})
The_Graph

In [30]:
me = Trader()
buy_open = []
sell_close = []
sell_open = []
buy_close = []
for i in range(1,len(xaus)):
    price = xaus[i]
    time = dates[i]
    pre_price = xaus[i-1]
    if model.predict(test_input.iloc[i:i+1,:])[0][0] > 0.5:
        # 觉得要涨
        if me.short_position > 0:
            # 如果有空仓，赶紧平
            me.buy_close(me.short_position,price,time)
            buy_close.append((time,price))
            print("I'am running away, bye!")
        else:
            print('I have no short position, take it easy.')
        # 然后开一仓
        me.buy_open(5, price, time)
        buy_open.append((time,price))
    else: # 反过来，如果觉得要跌了
        if me.long_position>0:
            # 如果有多仓，赶紧平
            me.sell_close(me.long_position,price,time)
            sell_close.append((time,price))
            print("I'am running away, bye!")
        else:
            print("I have no long position, take it easy")
        # 然后开新仓
        me.sell_open(5, price, time)
        sell_open.append((time,price))
    # if price>pre_price:
    #     if price<pre_price*0.98: # 如果涨幅达2%，平空仓，不开空仓，如果涨了但是不足2%，开多仓，不用平空仓
    #         print(str(time)+' price went up too high, stop shorting!')
    #         if me.short_position > 0:
    #             me.buy_close(me.short_position,price,time)
    #             print("I'am running away, bye!")
    #         else:
    #             print('I have no short position, take it easy.')
    #     else:
    #         me.buy_open(5, price, time)
    # else:
    #     if price>pre_price*1.02:# 如果跌幅达2%，平多仓，不开空仓，如果跌了但是不足2%，开空仓，不用平多仓
    #         print(str(time)+' price went up too low, stop longing!')
    #         if me.long_position>0:
    #             me.sell_close(me.long_position,price,time)
    #             print("I'am running away, bye!")
    #         else:
    #             print("I have no long position, take it easy")
    #     else:
    #         me.sell_open(5, price, time)
    me.settle(price, time)

，保证金账户余额为1016675.0000000014，所需保证金总额为5000
Balance changed: -800.0
This day is over.
defaultUser在价格1284.2平空仓5手，现仓位为多0，空0，保证金账户余额为1015875.0000000014，所需保证金总额为0
I'am running away, bye!
defaultUser在价格1284.2开多仓5手，现仓位为多5，空0，保证金账户余额为1015825.0000000014，所需保证金总额为5000
Balance changed: -3250.0
This day is over.
I have no short position, take it easy.
defaultUser在价格1279.55开多仓5手，现仓位为多10，空0，保证金账户余额为1012525.0000000014，所需保证金总额为10000
Balance changed: -2325.0
This day is over.
I have no short position, take it easy.
defaultUser在价格1282.1开多仓5手，现仓位为多15，空0，保证金账户余额为1010150.0000000014，所需保证金总额为15000
Balance changed: 2550.0
This day is over.
I have no short position, take it easy.
defaultUser在价格1279.8开多仓5手，现仓位为多20，空0，保证金账户余额为1012650.0000000014，所需保证金总额为20000
Balance changed: -3450.0
This day is over.
defaultUser在价格1283.7平多仓20手，现仓位为多0，空0，保证金账户余额为1009200.0000000014，所需保证金总额为0
I'am running away, bye!
defaultUser在价格1283.7开空仓5手，现仓位为多0，空5，保证金账户余额为1009150.0000000014，所需保证金总额为5000
Balance changed: 7800.0
This day is over.
I 

In [31]:
The_Graph.add_data_set(buy_open, series_type='scatter',name='Buy_open', marker={'enabled': True, 'radius':5, 'symbol':'triangle'},color="red")
The_Graph.add_data_set(sell_close, series_type='scatter',name='Sell_close', marker={'enabled': True, 'radius':5, 'symbol':'triangle-down'},color="red")
The_Graph.add_data_set(sell_open, series_type='scatter',name='Sell_open', marker={'enabled': True, 'radius':5, 'symbol':'triangle-down'},color="green")
The_Graph.add_data_set(buy_close, series_type='scatter',name='Buy_close', marker={'enabled': True, 'radius':5, 'symbol':'triangle'},color="green")
The_Graph

In [32]:
me.records.to_csv('records_4nn_1.csv', index=False)